# Kwantisering en bemonstering

De meeste signalen in het leven zijn continu: drukgolven die zich door lucht voortplanten, chemische reacties, lichaamsbeweging. Om deze continue signalen te verwerken, moeten ze echter worden omgezet naar digitale representaties via een analoog-naar-digitaal converter (ADC). Een digitaal signaal verschilt op twee  manieren van zijn continue tegenhanger:
- Het wordt **bemonsterd** op specifieke **tijdstappen**. Geluid wordt bijvoorbeeld vaak bemonsterd op 44,1 kHz (of eens per 0,023 milliseconden).
- Het wordt **gekwantiseerd** op specifieke **spanningsniveaus**. Bijvoorbeeld, op de Arduino Uno heeft de microcontroller een 10-bit ADC, dus een binnenkomende, continue spanningsinvoer kan worden gedigitaliseerd in stappen van $\frac{5V}{2^{10}}=4.88 mV$.

In deze les zullen we audiogegevens gebruiken als ons primaire signaal. Geluid is een prachtig medium om te leren omdat we zowel het signaal kunnen **visualiseren** als **horen**. Bedenk dat een microfoon reageert op luchtdruk golven. We zullen deze golfvormen plotten, manipuleren en vervolgens afspelen. We raden aan om je koptelefoon aan te sluiten, zodat je echt de verschillen in de verschillende audiofragmenten kunt horen.

## Afhankelijkheden

Voor dit notebook is [LibROSA](https://librosa.github.io/librosa/index.html) vereist - een python pakket voor muziek- en audioanalyse. Dit wordt mee geinstalleerd via dit notebook.


In [ ]:
import sys
!{sys.executable} -m pip install librosa

## Gebruikte bibliotheken

Hieronder staat heel wat Python code waarmee externe bibliotheken worden ingelezen. Klik gewoon 2x op 'play' zodat de notebook deze code zal uitvoeren. Je gaat er niet veel van merken, maar deze stap is nodig.

In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt # matplot lib is the premiere plotting lib for Python: https://matplotlib.org/
# for creating a responsive plot
%matplotlib widget
import numpy as np # numpy is the premiere signal handling library for Python: http://www.numpy.org/
import scipy as sp # for signal processing
from scipy import signal
import random

In [ ]:
def plot_signal_to_axes(ax, s, sampling_rate, title=None, signal_label=None, marker=None):
    '''Plots a sine wave s with the given sampling rate
    
    Parameters:
    ax: matplot axis to do the plotting
    s: numpy array
    sampling_rate: sampling rate of s
    title: chart title
    signal_label: the label of the signal
    '''
    ax.plot(s, label=signal_label, marker=marker, alpha=0.9)
    ax.set(xlabel="Samples")
    ax.set(ylabel="Amplitude")
    if signal_label is not None:
        ax.legend()

    # we use y=1.14 to make room for the secondary x-axis
    # see: https://stackoverflow.com/questions/12750355/python-matplotlib-figure-title-overlaps-axes-label-when-using-twiny
    if title is not None:
        ax.set_title(title, y=1.1)
    
    ax.grid()

    # add in a secondary x-axis to draw the x ticks as time (rather than samples)
    ax2 = ax.twiny()
    ax2.set_xlim(ax.get_xlim())
    
    ax_ticks = ax.get_xticks()[1:-1]
    ax2_tick_labels = ax.get_xticks()[1:-1] / sampling_rate

    num_samples_shown = ax.get_xlim()[1] - ax.get_xlim()[0]
    time_shown = num_samples_shown / sampling_rate
    if time_shown < 1:
        ax2.set_xlabel("Time (ms)")
        # format with 'g' causes insignificant trailing zeroes to be removed
        # https://stackoverflow.com/a/2440708 but also uses scientific notation, oh well!
        ax2_tick_labels = [f"{x * 1000:.1f}" for x in ax2_tick_labels]
    else:
        ax2.set_xlabel("Time (secs)")
        ax2_tick_labels = ['{:.2f}'.format(x) for x in ax2_tick_labels]

    ax2.set_xticks(ax_ticks)
    ax2.set_xticklabels(ax2_tick_labels)

def plot_signal(s, sampling_rate, title = None, xlim_zoom = None, highlight_zoom_area = True):
    '''Plots time-series data with the given sampling_rate and xlim_zoom'''
    
    plot_title = title
    if plot_title is None:
        plot_title = f"Sampling rate: {sampling_rate} Hz"

    if xlim_zoom == None:
        fig, axes = plt.subplots(1, 1, figsize=(15,6))
        
        plot_signal_to_axes(axes, s, sampling_rate, plot_title)
        return (fig, axes)
    else:
        #fig, axes = plt.subplots(1, 2, figsize=(15,6), sharey=True, gridspec_kw={'width_ratios': [2, 1]})
        fig, axes = plt.subplots(1, 2, figsize=(12,6))
        plot_signal_to_axes(axes[0], s, sampling_rate, plot_title)
        
        # if(xlim_zoom == None):
        #     xlim_zoom = get_random_xzoom(len(audio_data), 0.1)
        
        if highlight_zoom_area:
            # yellow highlight color: color='#FFFBCC'
            axes[0].axvspan(xlim_zoom[0], xlim_zoom[1], color='orange', alpha=0.3)
            
        axes[1].set_xlim(xlim_zoom)
        zoom_title = f"Signal zoomed: {int(xlim_zoom[0])} - {int(xlim_zoom[1])} samples"
        plot_signal_to_axes(axes[1], s, sampling_rate, zoom_title)
        axes[1].set_ylabel(None)
        fig.tight_layout()
        return (fig, axes)

def create_sine_wave(freq, sampling_rate, total_time_in_secs = None, return_time = False):
    '''Creates a sine wave with the given frequency, sampling rate, and length'''
    
    # if the total time in secs is None, then return one period of the wave
    if total_time_in_secs is None:
        total_time_in_secs = 1 / freq

    # Create an array from 0 to total_time_in_secs * sampling_rate (and then divide by sampling
    # rate to get each time_step)
    time = np.arange(total_time_in_secs * sampling_rate) / sampling_rate
    
    # Could also generate this signal by:
    # time = np.linspace(0, total_time_in_secs, int(total_time_in_secs * sampling_rate), endpoint=False)

    sine_wave = np.sin(2 * np.pi * freq * time)

    # or, once the sample is made:
    # time = np.linspace(0, len(s) / sampling_rate, num=len(s))

    if return_time is False:
        return sine_wave
    else:
        return (time, sine_wave)


# Deel 1 - Kwantisatie

Kwantisatie verwijst naar het proces van het transformeren van een analoog signaal, dat een continue reeks waarden heeft, naar een digitaal signaal, dat een discrete set heeft. Zie de figuur hieronder uit het artikel over [Kwantisatie](https://en.wikipedia.org/wiki/Quantization_(signal_processing)) op Wikipedia.

| 2-bits Kwantisatie | 3-bits Kwantisatie |
| :-----: | :-----: |
| ![2-bit resolution](https://upload.wikimedia.org/wikipedia/commons/b/b1/2-bit_resolution_analog_comparison.png) | ![3-bit resolution](https://upload.wikimedia.org/wikipedia/commons/b/b7/3-bit_resolution_analog_comparison.png) |
| 2-bits resolutie kwantiseert het analoge signaal in vier niveaus ($2^{2}$) | 3-bits resolutie kwantiseert in acht niveaus ($2^{3}$) |

De ATmega328 op de Arduino Uno heeft bijvoorbeeld een 10-bits analoog-naar-digitaal (ADC) converter terwijl de ESP32 een 12-bits ADC heeft. Omdat de ATmega328 op 5V werkt, is de stapgrootte van de ADC $\frac{5V}{2^{10}} = 4.88mV$. Dit is de kleinste waarneembare verandering die je kunt observeren op de analoge invoerpinnen van de Uno. In contrast, de ESP32 werkt op 3.3V en heeft een hogere bitresolutie (12 bits), dus de ADC heeft veel fijnere discretisaties: $\frac{3.3V}{2^{12}} = 0.806mV$—ongeveer, zes keer meer precisie dan de Uno!

## Karakterisering van kwantisatiefout

Een gedigitaliseerde sample kan een maximale fout hebben van de helft van de discretisatiestap (*d.w.z.,* ±½ van de Least Significant Bit (LSB)). Waarom? Omdat wanneer we een analoog waarde omzetten naar een digitale, we naar het dichtstbijzijnde gehele getal afronden. Stel je een spanningsignaal van 0.2271V voor op een Uno's analoge invoerpin, dit ligt bijna halverwege tussen de stappen 0.2246V en 0.2295V, wat zou resulteren in een fout van $\frac{4.89mV}{2}$ (en wordt omgezet naar 47 of 48 via `analogRead` van Arduino).

## Hoe beïnvloedt kwantisatie audio signalen?
Voor de onderstaande voorbeelden werken we met vooraf gedigitaliseerde audiogolven bemonsterd op 44.1kHz en gekwantiseerd op 16-bits. Dus, hoewel het geen echt continue sample is (natuurlijk niet, het is al een digitaal signaal!), zullen we het losjes als zodanig behandelen. En we zullen "downsamplen" om de effecten van kwantisatieniveaus en bemonsteringssnelheden te onderzoeken.

Laten we een initiële 16-bit, 44.1kHz geluidsgolfvorm van iemand die het woord "Hallo" zegt, laden, visualiseren en afspelen.

In [ ]:
sampling_rate, audio_data_16bit = sp.io.wavfile.read('HumanVoice-Hello_16bit_44.1kHz_mono.wav')

print(f"Sampling rate: {sampling_rate} Hz")
print(f"Number of channels = {len(audio_data_16bit.shape)}")
print(f"Total samples: {audio_data_16bit.shape[0]}")

if len(audio_data_16bit.shape) == 2:
    # convert to mono
    audio_data_16bit = convert_to_mono(audio_data_16bit)
    
length_in_secs = audio_data_16bit.shape[0] / sampling_rate
print(f"length = {length_in_secs}s")
print(audio_data_16bit)
quantization_bits = 16
print(f"{quantization_bits}-bit audio ranges from -{2**(quantization_bits - 1)} to {2**(quantization_bits - 1) - 1}")
print(f"Max value: {np.max(audio_data_16bit)} Avg value: {np.mean(audio_data_16bit):.2f}")

# We'll highlight and zoom in on the orange part of the graph controlled by xlim_zoom
xlim_zoom = (11000, 12500) # you may want to change this depending on what audio file you have loaded
plot_signal(audio_data_16bit, sampling_rate, quantization_bits, xlim_zoom = xlim_zoom)
ipd.Audio(audio_data_16bit, rate=sampling_rate)


### 8-bit kwantisatie

We kunnen de 16-bits audio omzetten naar andere kwantisatieniveaus om te zien en te horen hoe kwantisatie de kwaliteit beïnvloedt.

We beginnen met een 8-bit kwantisatie; pas de parameter aan in de volgende code blok, voer de daaropvolgende codeblok uit en bekijk het resultaat. Vergelijk ook telkens met de oorspronkelijke 16-bit versie hierboven.

In [ ]:
# define the quantization level here
quantization_bits = 8

In [ ]:
# Convert to float
audio_data_float = audio_data_16bit / 2**16 # 16 bit audio

# Let's try with n-bit audio
audio_data_nbit = audio_data_float * 2**quantization_bits
audio_data_nbit = audio_data_nbit.astype(int)
print(audio_data_nbit)
print(f"{quantization_bits}-bit audio ranges from -{2**(quantization_bits - 1)} to {2**(quantization_bits - 1) - 1}")
print(f"Max value: {np.max(audio_data_nbit)} Avg value: {np.mean(audio_data_nbit):.2f}")

plot_signal(audio_data_nbit, sampling_rate, quantization_bits, xlim_zoom = xlim_zoom)
ipd.Audio(audio_data_nbit, rate=sampling_rate)

**Vragen** (vul hieronder je antwoord in)

- Welk bereik (minimum en maximum van de amplitude) heeft het geluidsignaal na kwantisatie op 8 bits?

- Waar komen deze waarden vandaan?

- Kijk goed naar de golfvorm, kun je verschillen opmerken met 16-bits audio? 

- En hoe zit het wanneer je luistert naar de 8-bits versus de 16-bits versie?

### 6-bit kwantisatie
Hoe zit het met 6-bits? Pas de parameter quantization_bits aan tot 6 om dit uit te proberen.

**Vragen**
- Hoor je degradaties in het signaal? Omschrijf wat je hoort.
  
- En kan je al gekwantiseerde stappen zien in de ingezoomde golfvorm?

### 4-bit kwantisatie
Pas nu de parameter quantization_bits aan tot 4.

**Vragen**
- Hoor je degradaties in het signaal? Omschrijf wat je hoort.
  
- En wat zie je in de ingezoomde golfvorm?

## Experimenteer met een ander geluidsfragment

We laden nu een ander geluidsfragment in en onderzoeken de invloed van de kwantisatie. We letten nu vooral op het verschil in kwaliteit tussen luide en stille delen van het geluidssignaal.

Zoom in op een deel van het signaal waarin zowel luide als zachte stukken zitten. Inzoomen kan je met het vierkantje linksonder in het menu van de grafiek. Doe dit voor verschillende waarden van quantization_bits. Begin met 8 en ga zo naar 6, 4 en 3 bits.

In [ ]:
# Verander deze parameter om het aantal bits te wijzigen
quantization_bits = 8 # change this and see what happens!
# --------------------------------------

# Change this wave file to any 16-bit audio sample
your_sound_file = 'Guitar_MoreThanWords_16bit_44.1kHz_stereo.wav'
your_sampling_rate, your_audio_data_16_bit = sp.io.wavfile.read(your_sound_file)

print(f"Sampling rate: {your_sampling_rate} Hz")
print(f"Number of channels = {len(your_audio_data_16_bit.shape)}")
print(f"Total samples: {your_audio_data_16_bit.shape[0]}")

if len(your_audio_data_16_bit.shape) == 2:
    # convert to mono
    print("Converting stereo audio file to mono")
    your_audio_data_16_bit = your_audio_data_16_bit.sum(axis=1) / 2

# Convert to float
your_audio_data_float = your_audio_data_16_bit / 2**16 # 16 bit audio

your_audio_data_quantized = your_audio_data_float * 2**quantization_bits
your_audio_data_quantized = your_audio_data_quantized.astype(int)
print(your_audio_data_quantized)
print(f"{quantization_bits}-bit audio ranges from -{2**(quantization_bits - 1)} to {2**(quantization_bits - 1) - 1}")
print(f"Max value: {np.max(your_audio_data_quantized)} Avg value: {np.mean(your_audio_data_quantized):.2f}")

xlim_zoom = (45000, 50000) # make sure to change the zoom range too
plot_signal(your_audio_data_16_bit, sampling_rate, 16, xlim_zoom = xlim_zoom)
plot_signal(your_audio_data_quantized, sampling_rate, quantization_bits, xlim_zoom = xlim_zoom)
ipd.Audio(your_audio_data_quantized, rate=sampling_rate)

**Vragen**

- Als je inzoomt op een gebied met luide en zachte stukken, welk verschil zie je dan tussen beide?

- Bij een kwantisatie naar 6 bits, hoeveel verschillende waardes zie je voor het zachte deel van het signaal (met een amplitude van 1)? 
  
- En voor het luide deel van het signaal (amplitude 10)?


# Conclusie

Een kleiner signaal (zachte deel van de muziek) gebruikt minder digitale waardes dan een groter signaal (het luide deel). 
Een klein analoog signaal omzetten naar een digitaal signaal levert dus een kwalitatief minder goed digitaal signaal op dan een groot analoog signaal. 

Een Analoog Digitaal Convertor van 5V en N bits zet een analoog signaal van 5V om in $2^{N}$ niveaus, zoals we in de grafieken hebben gezien.  
We hebben minstens 1 niveau per millivolt nodig om het signaal te kunnen detecteren.
    
**vragen**

- Ons oog produceert een signaal van ongeveer 1 mV. Hoeveel bits denk je nodig te hebben om dit signaal te digitaliseren?  

- Onze ADC is beperkt tot 10 bits. Wat moet je doen? 

- Wat is je conclusie uit dit experiment?

# Deel 2 - Bemonstering

<img src="https://upload.wikimedia.org/wikipedia/commons/5/50/Signal_Sampling.png" width="500">
<center>Een continu signaal (groen) wordt elke $\frac{1}{T}$ Hz bemonsterd (blauw). Bron: <a href="https://en.wikipedia.org/wiki/Sampling_(signal_processing)">Wikipedia</a></center>



Naast kwantisatie is de andere belangrijke factor bij het digitaliseren van een signaal de snelheid waarmee het analoge signaal wordt bemonsterd (of vastgelegd). Hoe vaak moet je een signaal monsteren om het perfect te reconstrueren?

## Nyquist bemonsteringstheorema
Het antwoord kan je verrassen en omvat een van de meest fundamentele (en interessante) stellingen in signaalverwerking: het Nyquist-bemonsteringstheorema, die stelt dat een continu signaal gereconstrueerd kan worden zolang er twee monsters per periode zijn voor de hoogste frequentiecomponent in het onderliggende signaal.

Dat wil zeggen, voor een perfecte reconstructie moet onze digitale bemonsteringsfrequentie $f_s$ minstens twee keer zo snel zijn als de snelste frequentie in ons continue signaal: $f_s = 2 * max(analog_{freq})$.

Bijvoorbeeld, stel dat we een analoog signaal hebben bestaande uit frequenties tussen 0 en 2.000 Hz. Om dit signaal goed te digitaliseren, moeten we monsteren bij $2 * 2.000Hz$. Dus, $f_s$ moet 4.000Hz zijn.

Stel je nu voor dat het snelste waarmee onze digitizer kan bemonsteren 6.000 Hz is: welk frequentiebereik kunnen we dan goed vastleggen? Omdat we minimaal twee monsters per periode nodig hebben voor een juiste reconstructie, kunnen we alleen signalen vastleggen die veranderen met een frequentie van 0 tot maximaal 3.000Hz. Deze limiet van 3.000Hz wordt de Nyquist-frequentie of Nyquist-limiet genoemd: het is $\frac{1}{2}$ van de bemonsteringsfrequentie $f_s$.

Voor veel toepassingen met betrekking tot Mens-Computerinteractie is bemonstering bij 4kHz meer dan voldoende. Dit maakt analyse van elk signaal tussen 0-2kHz mogelijk. Menselijke beweging - wandelbeweging, ledemaatbeweging, vingerbewegingen, enzovoort - verandert gewoon niet zo snel. Zelfs elektro-encefalogrammen (EEG), die elektrische activiteit in de hersenen meten, worden vaak bemonsterd bij 500-1000Hz. Echter, voor het opnemen van geluid (mensen kunnen horen tussen ~0-20kHz), zijn snellere bemonsteringsfrequenties nodig.


## Aliasing

Wat gebeurt er als we een signaal monsteren met frequentiecomponenten groter dan de Nyquist-limiet? We krijgen [aliasing](https://en.wikipedia.org/wiki/Aliasing#Sampling_sinusoidal_functions) - een probleem waarbij de hogere frequentiecomponenten van een signaal (die groter zijn dan de Nyquist-limiet) verschijnen als lagere frequentiecomponenten. Zoals Smith opmerkt, "net zoals een crimineel een aangenomen naam of identiteit kan aannemen (een alias), neemt de sinusgolf een andere frequentie aan die niet van hemzelf is." En misschien nog kwalijker, er is niets in de bemonsterde gegevens dat suggereert dat er aliasing heeft plaatsgevonden: "de sinusoïde heeft zijn ware identiteit volledig verborgen." Zie figuur hieronder.

<img src="NyquistSamplingTheoremAndAliasing_1500w.png"/>

### Aliasing voorbeeld

Laten we eens naar een voorbeeld kijken. Hier zullen we vier signalen bemonsteren met een bemonsteringsfrequentie van 50Hz: `signaal1 = 5Hz`, `signaal2 = 10Hz`, `signaal3 = 20Hz`, en `signaal4 = 60Hz`. Alleen `signaal4` bevindt zich boven onze Nyquist-limiet, die $\frac{1}{2} * 50Hz = 25Hz$ is. Wat zal er gebeuren?

De "monsters" worden weergegeven als verticale lijnen met vierkante rechthoekige markeringen. Wat observeer je? Let goed op `signaal4`...

In [ ]:
total_time_in_secs = 0.2

# Create our "real-world" continuous signals (which is obviously not possible on a digital computer, so we fake it)
real_world_continuous_speed = 10000
real_world_signal1_freq = 5
time, real_world_signal1 = create_sine_wave(real_world_signal1_freq, real_world_continuous_speed, 
                                               total_time_in_secs, return_time = True)

real_world_signal2_freq = 10
real_world_signal2 = create_sine_wave(real_world_signal2_freq, real_world_continuous_speed, 
                                               total_time_in_secs)

real_world_signal3_freq = 20
real_world_signal3 = create_sine_wave(real_world_signal3_freq, real_world_continuous_speed, 
                                               total_time_in_secs)

real_world_signal4_freq = 60
real_world_signal4 = create_sine_wave(real_world_signal4_freq, real_world_continuous_speed, 
                                               total_time_in_secs)

# Create the sampled versions of these continuous signals
resample_factor = 200 # should be an integer
sampled_time = time[::resample_factor]
sampled_signal1 = real_world_signal1[::resample_factor]
sampled_signal2 = real_world_signal2[::resample_factor]
sampled_signal3 = real_world_signal3[::resample_factor]
sampled_signal4 = real_world_signal4[::resample_factor]
sampling_rate = real_world_continuous_speed / resample_factor
print(f"Sampling rate: {sampling_rate} Hz")

# Visualize the sampled versions
fig, axes = plt.subplots(4, 1, figsize=(10,13))
axes[0].plot(time, real_world_signal1)
axes[0].axhline(0, color="gray", linestyle="-", linewidth=0.5)
axes[0].plot(sampled_time, sampled_signal1, linestyle='None', alpha=0.8, marker='s', color='black')
axes[0].vlines(sampled_time, ymin=0, ymax=sampled_signal1, linestyle='-.', alpha=0.8, color='black')
axes[0].set_ylabel("Amplitude")
axes[0].set_xlabel("Time (secs)")
axes[0].set_title(f"{real_world_signal1_freq}Hz signal sampled at {sampling_rate}Hz")

axes[1].plot(time, real_world_signal2)
axes[1].axhline(0, color="gray", linestyle="-", linewidth=0.5)
axes[1].plot(sampled_time, sampled_signal2, linestyle='None', alpha=0.8, marker='s', color='black')
axes[1].vlines(sampled_time, ymin=0, ymax=sampled_signal2, linestyle='-.', alpha=0.8, color='black')
axes[1].set_ylabel("Amplitude")
axes[1].set_xlabel("Time (secs)")
axes[1].set_title(f"{real_world_signal2_freq}Hz signal sampled at {sampling_rate}Hz")

axes[2].plot(time, real_world_signal3)
axes[2].axhline(0, color="gray", linestyle="-", linewidth=0.5)
axes[2].plot(sampled_time, sampled_signal3, linestyle='None', alpha=0.8, marker='s', color='black')
axes[2].vlines(sampled_time, ymin=0, ymax=sampled_signal3, linestyle='-.', alpha=0.8, color='black')
axes[2].set_ylabel("Amplitude")
axes[2].set_xlabel("Time (secs)")
axes[2].set_title(f"{real_world_signal3_freq}Hz signal sampled at {sampling_rate}Hz")

axes[3].plot(time, real_world_signal4)
axes[3].axhline(0, color="gray", linestyle="-", linewidth=0.5)
axes[3].plot(sampled_time, sampled_signal4, linestyle='None', alpha=0.8, marker='s', color='black')
axes[3].vlines(sampled_time, ymin=0, ymax=sampled_signal4, linestyle='-.', alpha=0.8, color='black')
axes[3].set_ylabel("Amplitude")
axes[3].set_xlabel("Time (secs)")
axes[3].set_title(f"{real_world_signal4_freq}Hz signal sampled at {sampling_rate}Hz")

fig.tight_layout(pad = 3.0)

Laten we eens wat beter kijken naar signaal2 = 10Hz en signaal4 = 60Hz.

Kun je het zien? Het 60Hz signaal wordt bemonsterd als een 10Hz signaal. En zodra het signaal is gedigitaliseerd (zoals hier), zou er geen manier zijn om het verschil te zien tussen een werkelijk 10Hz signaal en een gealieaseerd signaal!

Waarom?

Kijk naar de grafieken, bij het eerste monster beginnen beide sinusoiden net; echter, bij het volgende monster heeft de 60Hz sinusoid bijna één volledige periode voltooid!


## Hoe beïnvloeden bemonsteringsfrequenties de geluidskwaliteit?

Hieronder bemonsteren we een menselijke stem van 44,1 kHz naar: 22,5 kHz, 11.025 Hz ... 441 Hz. Voor elke bemonstering visualiseren we de originele golfvorm van 44,1 kHz en zijn bijbehorende bemonsterde tegenhanger.

Wat valt je op?

In [ ]:
# feel free to change this to some other 44.1kHz sound file
sound_file = 'HumanVoice-Hello_16bit_44.1kHz_mono.wav' 
sampling_rate, audio_data_44100 = sp.io.wavfile.read(sound_file)

print(f"Sampling rate: {sampling_rate} Hz")
print(f"Number of channels = {len(audio_data_44100.shape)}")
print(f"Total samples: {audio_data_44100.shape[0]}")

if len(audio_data_44100.shape) == 2:
    # convert to mono
    print("Converting stereo audio file to mono")
    audio_data_44100 = audio_data_44100.sum(axis=1) / 2
    
quantization_bits = 16
xlim_zoom = (11500, 12500)

plot_signal(audio_data_44100, sampling_rate, quantization_bits, xlim_zoom = xlim_zoom)
ipd.Audio(audio_data_44100, rate=sampling_rate)


In [ ]:
resample_factor = 10
new_sampling_rate = sampling_rate / resample_factor
audio_data_resampled = audio_data_44100[::resample_factor]
resample_xlim_zoom = (xlim_zoom[0] / resample_factor, xlim_zoom[1] / resample_factor)

print(f"Sampling rate: {new_sampling_rate} Hz")
print(f"Number of channels = {len(audio_data_resampled.shape)}")
print(f"Total samples: {audio_data_resampled.shape[0]}")

plot_signal(audio_data_44100, sampling_rate, quantization_bits, xlim_zoom = xlim_zoom)
ipd.Audio(audio_data_resampled, rate=new_sampling_rate)

## Bronnen
- [Chapter 3: ADC and DAC](http://www.dspguide.com/ch3.htm), The Scientist and Engineer's Guide to Digital Signal Processing, Steven W. Smith, Ph.D.
- [Chapter 2: Signals in the Computer](http://faculty.washington.edu/stiber/pubs/Signal-Computing/Signal%20Computing.pdf), Signal Computing: Digital Signals in the Software Domain, Stiber, Stiber, and Larson, 2020
- [Notes on Music Information Retrieval](https://musicinformationretrieval.com/index.html)
- [D/A and A/D](https://youtu.be/cIQ9IXSUzuM), Monty Montgomery, YouTube

## Over dit Notebook

This Notebook was designed and written by Professor Jon E. Froehlich at the University of Washington along with feedback from students. It is made available freely online as an [open educational resource](https://en.wikipedia.org/wiki/Open_educational_resources) at the teaching website: https://makeabilitylab.github.io/physcomp/. 

The [website](https://github.com/makeabilitylab/physcomp), [Notebook code](https://github.com/makeabilitylab/signals), and [Arduino code](https://github.com/makeabilitylab/arduino) are all open source using the MIT license.

Het Notebook is aangepast en vertaald naar het Nederlands door Roel Truyen.
